# Tool Calling with LangChain (Runnable API)

## Learning Goals
- Define callable tools with clear input schemas.
- Drive tool selection from an LLM using **structured JSON outputs**.
- Compose a minimal **Runnable** pipeline (`|` and `.invoke`) that parses the LLM decision and dispatches the selected tool.
- Understand how this pattern relates to ReAct and paves the way for LangGraph.

This notebook corresponds to the *Tool Calling* sub-section and uses the modern **Runnable API** in LangChain.

In [4]:
# %load get_llm.py
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

# Load environment variables from .env
load_dotenv()

def get_llm(provider: str = "openai"):
    """
    Return a language model instance configured for either OpenAI or Ollama.

    This function centralizes the initialization of chat-based LLMs so that 
    notebooks and applications can switch seamlessly between cloud-based models 
    (OpenAI) and local models (Ollama).

    Parameters
    ----------
    provider : str, optional
        The backend provider to use. Options are:
        - "openai": returns a ChatOpenAI instance (requires OPENAI_API_KEY in .env).
        - "ollama": returns a ChatOllama instance (requires Ollama installed locally).
        Default is "openai".

    Returns
    -------
    langchain.chat_models.base.BaseChatModel
        A chat model instance that can be invoked with messages.

    Examples
    --------
    Initialize an OpenAI model (requires API key):

    >>> llm = get_llm("openai")
    >>> llm.invoke("Hello, how are you?")

    Initialize a local Ollama model (e.g., Gemma2 2B):

    >>> llm = get_llm("ollama")
    >>> llm.invoke("Summarize the benefits of reinforcement learning.")
    """
    if provider == "openai":
        return ChatOpenAI(
            model="gpt-4o-mini",  # can also be "gpt-4.1" or "gpt-4o"
            temperature=0
        )
    elif provider == "ollama":
        return ChatOllama(
            model="gemma2:2b",   # replace with any local model installed in Ollama
            temperature=0
        )
    else:
        raise ValueError("Unsupported provider. Use 'openai' or 'ollama'.")


In [5]:
# Setup
import json
from typing import Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain.schema import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# It is assumed that get_llm() is defined elsewhere (Notebook 2 helper)
llm = get_llm("openai")  # or get_llm("ollama")

## Step 1 - Define tools with clear input schemas
Two tools are defined: a **calculator** and a **dictionary**. Each tool has a minimal validation layer.

In [6]:
def calculator(expression: str) -> str:
    """Evaluate a simple Python arithmetic expression safely."""
    try:
        # Very constrained eval; in production use a proper expression parser
        return str(eval(expression, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Calculator error: {e}"

DICTIONARY = {
    "agent": "An autonomous problem-solving system.",
    "database": "An organized collection of structured information.",
    "token": "A basic unit of text used by language models.",
}

def define_word(word: str) -> str:
    return DICTIONARY.get(word.lower(), "Word not found")

## Step 2 - Specify the decision schema
A Pydantic model is used so the LLM is guided to return structured JSON that can be parsed reliably.

In [7]:
class ToolDecision(BaseModel):
    tool: Literal["calculator", "dictionary"] = Field(..., description="Tool to call")
    input: str = Field(..., description="Argument to pass to the selected tool")

def parse_json(text: str) -> ToolDecision:
    try:
        data = json.loads(text)
        return ToolDecision(**data)
    except Exception as e:
        raise ValueError(f"Invalid tool decision JSON: {e}\nRaw: {text}")

## Step 3 - Prompt the LLM to produce a valid JSON decision
A system prompt describes the tools and the required output format.

In [11]:
SYSTEM_INSTRUCTIONS = (
    "You are a tool-choosing assistant. "
    "Given a user query, choose exactly one tool and provide a valid JSON object.\n"
    "Available tools:\n"
    "- calculator: evaluate arithmetic expressions. Example: {{\"tool\": \"calculator\", \"input\": \"12*7\"}}\n"
    "- dictionary: provide short definitions. Example: {{\"tool\": \"dictionary\", \"input\": \"agent\"}}\n\n"
    "Respond with JSON only. No extra text."
)

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_INSTRUCTIONS),
    ("human", "{question}")
])

pipeline = (
    prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(parse_json)
)

## Step 4 - Dispatch the selected tool
A dispatcher maps `tool` → function. The entire flow is kept inside the Runnable chain for clarity.

In [12]:
TOOLBOX = {
    "calculator": calculator,
    "dictionary": define_word,
}

def dispatch(decision: ToolDecision) -> str:
    tool_fn = TOOLBOX.get(decision.tool)
    if tool_fn is None:
        return f"Unknown tool: {decision.tool}"
    return tool_fn(decision.input)

tool_runner = RunnableLambda(dispatch)

agent = pipeline | tool_runner

## Step 5 - Run examples
Two queries are tested: a math question and a definition request.

In [13]:
print(agent.invoke({"question": "What is 15 * 3?"}))
print(agent.invoke({"question": "Define: agent"}))

45
An autonomous problem-solving system.


# Difference between notebooks 2 and 4

## Notebook 2 - Minimal Agent

- Purpose: introduce the idea of an agent.

- Implementation style:

  - Direct calls to the LLM (llm.invoke) with a handcrafted system prompt.
  - Very simple string parsing (later improved to JSON parsing).
  - Tools (calculator, dictionary) are plain Python functions.

- Key learning point: the separation between the LLM as the brain (decides) and the agent as the body (executes).

- Analogy: like wiring components manually to show the principle.

## Notebook 4 (this one) - Tool Calling

- Purpose: show how LangChain’s modern Runnable API structures tool calling.

- Implementation style:

    - Uses ChatPromptTemplate + Runnable composition (| operator, .invoke).

    - Defines a Pydantic schema (ToolDecision) for structured outputs.

    - Explicit dispatcher mapping tool names → Python functions.

    - Enforces valid JSON and integrates error handling.

- Key learning point: this is a robust, extensible pattern, closer to what production LangChain agents do (and a stepping stone toward LangGraph).

- Analogy: like replacing a hand-wired circuit with a clean modular board.

### Reflection
- The LLM acts as a **router**, returning a JSON decision that identifies the tool and the input.
- The Runnable chain enforces **deterministic parsing** and **safe dispatch**.
- This is a minimal tool-calling agent; the same composition is expandable with additional tools and validation.
- The pattern connects naturally to **ReAct** (reason → action) and to **LangGraph** for multi-step control flows.

## Exercises
1. Add a **translator** tool (English → Portuguese) and integrate it into `TOOLBOX` and the system instructions.
2. Extend the JSON schema to include an optional `explanation` field that the LLM can fill with a short rationale; log it but ignore it for execution.
3. Replace `get_llm(\"openai\")` with `get_llm(\"ollama\")` and test with a local model (e.g., `gemma2:2b`).